## Real Time Testing

In [14]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
import traceback
from time import time
import os

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

In [15]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)
# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))
# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list
# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox
# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center
# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart
# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }
# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score
# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [16]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [17]:
import tensorflow as tf

# Load the model from the H5 file
model = tf.keras.models.load_model('../static-recognition/models/static_model.h5')

from static_files_io import readActionLabels

static_labels = readActionLabels()
static_labels

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 'airplane']

In [20]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)

fpsReader = FPS()

timeStats = []

try:

    keypointsHistory = deque()
    predictionHistory = deque()
    detectionThreshold = 1.0

    lastPredictionTime = time()
    predictionCooldown = 1

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Pose Detection
        detectionResults, frame = featureExtractionV3(
            handDetector, faceDetector, poseDetector, frame)

        detectionResults = np.expand_dims(
            detectionResults, axis=0)  # Reshape to (1, 138, 3)

        predictionResults = model.predict(
            x=detectionResults,
            verbose=0,
            use_multiprocessing=True,
            workers=4
        )[0]

        predCharacter = static_labels[np.argmax(predictionResults)]
        predAccuracy = predictionResults[np.argmax(predictionResults)]
        print(predictionResults)

        cv2.putText(frame, ', '.join(predCharacter), (15, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0), scale=5, thickness=5)

        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if (keyPressed == 27):
            raise Exception("Finished")


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()


[2.9625351e-05 1.0751271e-04 5.4197554e-03 4.9705623e-04 3.3307471e-04
 3.7153061e-03 4.9429911e-04 3.1022314e-04 1.9551141e-02 1.8575849e-03
 3.9911547e-04 1.9249331e-03 1.8373638e-03 2.1466747e-04 2.6086252e-03
 1.6686757e-03 2.0632450e-04 1.4109601e-03 1.9686176e-04 1.3864921e-02
 2.0265295e-03 1.0412721e-03 2.0732373e-04 2.3224938e-04 2.9046531e-03
 4.2731695e-02 5.0102440e-03 3.7479163e-03 4.0224422e-04 1.9706427e-01
 4.8020717e-02 5.5169749e-01 1.7398471e-02 1.3096267e-03 2.1260833e-02
 2.8560674e-02 1.9735770e-02]
[2.9388599e-05 1.1487956e-04 5.8009853e-03 3.0738467e-04 2.8140569e-04
 5.0639794e-03 4.4104242e-04 3.7996072e-04 1.5245591e-02 1.7896534e-03
 3.5466967e-04 2.7712048e-03 2.1149986e-03 2.8347244e-04 1.5155781e-03
 1.7223950e-03 1.3792200e-04 7.5976277e-04 2.0011328e-04 2.7976813e-02
 4.3098126e-03 2.5810055e-03 2.1402031e-04 2.3059228e-04 2.6781997e-03
 5.4523975e-02 4.7091921e-03 2.7030036e-03 3.1367518e-04 1.8846042e-01
 2.9867999e-02 5.6111628e-01 1.6093688e-02 7.88

In [19]:
timeStats[10:]

[0.10852622985839844,
 0.12427020072937012,
 0.12517285346984863,
 0.11609816551208496,
 0.14620399475097656,
 0.1326591968536377,
 0.13311386108398438,
 0.13241028785705566,
 0.12305307388305664,
 0.13984155654907227,
 0.1527407169342041,
 0.11153578758239746,
 0.13678407669067383,
 0.12055039405822754,
 0.1364138126373291,
 0.13200068473815918,
 0.1284034252166748,
 0.13766837120056152,
 0.12585806846618652,
 0.13331818580627441,
 0.13846278190612793,
 0.12427234649658203,
 0.13919782638549805,
 0.1368544101715088,
 0.1374368667602539,
 0.11771655082702637,
 0.1543869972229004,
 0.13257575035095215,
 0.11889386177062988,
 0.10781741142272949,
 0.13612842559814453,
 0.1035315990447998,
 0.10779547691345215,
 0.1077108383178711,
 0.10350990295410156,
 0.1283731460571289,
 0.10453963279724121,
 0.10856914520263672,
 0.13367843627929688,
 0.0986337661743164,
 0.1078038215637207,
 0.11859774589538574,
 0.11074948310852051,
 0.10859084129333496,
 0.10754203796386719,
 0.11053991317749023,
